In [1]:
import numpy as np
import pandas as pd
import math
import re

In [2]:
df_preds = pd.read_csv('./../Responses/Predictions/Argenx_GPT5_predictions.csv')

print(df_preds.shape)
df_preds.head()

(15, 4)


,input_column,matched_KF_column_1,matched_KF_column_2,matched_KF_column_3
0,ID,Employee ID,Manager Employee ID,Korn Ferry Reference Job Code
1,Gender,Gender,Executive/Non-Executive Indicator (E / N),Expat
2,Business Title,Employee Job Title,Department,Legal Entity / Business Unit
3,Country,Country,Employee Work Location / Zip/Postal Code,Legal Entity / Business Unit
4,argenx job level,Employee Grade / Band,Korn Ferry Reference Level (not required if ...,Korn Ferry Total Points


In [3]:
df_gt = pd.read_excel('./../Files/Ground_Truth/Argenx_mapped_to_ORGDC.xlsx')

print(df_gt.shape)
df_gt.head()

(9, 7)


,Input Header,Proposed Template Column,GroundTruth Template Column,Is_Correct,Stage,Confidence_Level,Decision
0,Gender,GENDER,GENDER,True,1,auto_accept,Mapped
1,Country,COUNTRY,COUNTRY,True,1,auto_accept,Mapped
2,Currency,CURRENCY,CURRENCY,True,1,auto_accept,Mapped
3,Allowance - Car,CAR ALLOWANCE (ANNUAL FULL-TIME EQUIVALENT),CAR ALLOWANCE (ANNUAL FULL-TIME EQUIVALENT),True,1,auto_accept,Mapped
4,Equity Amount - Annual,BASIC PAYMENTS (ANNUAL FULL-TIME EQUIVALENT),ACTUAL FAIR VALUE OF TOTAL LTI GRANTED,False,3,auto_accept,Mapped


In [4]:
# wherever Input Headers match, get the corresponding GT Template IDs
df_merged = pd.merge(df_preds, df_gt[['Input Header', 'GroundTruth Template Column']], left_on='input_column', right_on='Input Header', how='left')

print(df_merged.shape)
df_merged.head()

(15, 6)


,input_column,matched_KF_column_1,matched_KF_column_2,matched_KF_column_3,Input Header,GroundTruth Template Column
0,ID,Employee ID,Manager Employee ID,Korn Ferry Reference Job Code,ID,EMPLOYEE ID
1,Gender,Gender,Executive/Non-Executive Indicator (E / N),Expat,Gender,GENDER
2,Business Title,Employee Job Title,Department,Legal Entity / Business Unit,Business Title,EMPLOYEE JOB TITLE
3,Country,Country,Employee Work Location / Zip/Postal Code,Legal Entity / Business Unit,Country,COUNTRY
4,argenx job level,Employee Grade / Band,Korn Ferry Reference Level (not required if ...,Korn Ferry Total Points,NaN,NaN


In [5]:
df_merged_notNaN = df_merged[~df_merged['GroundTruth Template Column'].isna()]
print(df_merged_notNaN.shape)
df_merged_notNaN.head()

(9, 6)


,input_column,matched_KF_column_1,matched_KF_column_2,matched_KF_column_3,Input Header,GroundTruth Template Column
0,ID,Employee ID,Manager Employee ID,Korn Ferry Reference Job Code,ID,EMPLOYEE ID
1,Gender,Gender,Executive/Non-Executive Indicator (E / N),Expat,Gender,GENDER
2,Business Title,Employee Job Title,Department,Legal Entity / Business Unit,Business Title,EMPLOYEE JOB TITLE
3,Country,Country,Employee Work Location / Zip/Postal Code,Legal Entity / Business Unit,Country,COUNTRY
5,Currency,Currency,Country,Basic Payments (Annual full-time equivalent),Currency,CURRENCY


In [6]:
# calculate col1 acc, col2 acc, and col3 acc
# after making strings lowercase and stripping whitespace
df_merged_notNaN['matched_KF_column_1'] = df_merged_notNaN['matched_KF_column_1'].str.lower().str.strip()
df_merged_notNaN['matched_KF_column_2'] = df_merged_notNaN['matched_KF_column_2'].str.lower().str.strip()
df_merged_notNaN['matched_KF_column_3'] = df_merged_notNaN['matched_KF_column_3'].str.lower().str.strip()
df_merged_notNaN['GroundTruth Template Column'] = df_merged_notNaN['GroundTruth Template Column'].str.lower().str.strip()

df_merged_notNaN.head()

/var/folders/8b/nppjnl6j1xq3tmzf0rzzd8sr0000gp/T/ipykernel_54947/2065101339.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_notNaN['matched_KF_column_1'] = df_merged_notNaN['matched_KF_column_1'].str.lower().str.strip()
/var/folders/8b/nppjnl6j1xq3tmzf0rzzd8sr0000gp/T/ipykernel_54947/2065101339.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_notNaN['matched_KF_column_2'] = df_merged_notNaN['matched_KF_column_2'].str.lower().str.strip()
/var/folders/8b/nppjnl6j1xq3tmzf0rzzd8sr0000

,input_column,matched_KF_column_1,matched_KF_column_2,matched_KF_column_3,Input Header,GroundTruth Template Column
0,ID,employee id,manager employee id,korn ferry reference job code,ID,employee id
1,Gender,gender,executive/non-executive indicator (e / n),expat,Gender,gender
2,Business Title,employee job title,department,legal entity / business unit,Business Title,employee job title
3,Country,country,employee work location / zip/postal code,legal entity / business unit,Country,country
5,Currency,currency,country,basic payments (annual full-time equivalent),Currency,currency


In [7]:
# remove '\n' from GroundTruth Template Column
df_merged_notNaN['GroundTruth Template Column'] = df_merged_notNaN['GroundTruth Template Column'].str.replace('\n', ' ')
df_merged_notNaN.head()

/var/folders/8b/nppjnl6j1xq3tmzf0rzzd8sr0000gp/T/ipykernel_54947/1942819303.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_notNaN['GroundTruth Template Column'] = df_merged_notNaN['GroundTruth Template Column'].str.replace('\n', ' ')


,input_column,matched_KF_column_1,matched_KF_column_2,matched_KF_column_3,Input Header,GroundTruth Template Column
0,ID,employee id,manager employee id,korn ferry reference job code,ID,employee id
1,Gender,gender,executive/non-executive indicator (e / n),expat,Gender,gender
2,Business Title,employee job title,department,legal entity / business unit,Business Title,employee job title
3,Country,country,employee work location / zip/postal code,legal entity / business unit,Country,country
5,Currency,currency,country,basic payments (annual full-time equivalent),Currency,currency


In [8]:
print(df_merged_notNaN.iloc[0]['matched_KF_column_1'])
print(df_merged_notNaN.iloc[0]['GroundTruth Template Column'])
print(df_merged_notNaN.iloc[0]['matched_KF_column_1'] == df_merged_notNaN.iloc[0]['GroundTruth Template Column'])

employee id
employee id
True


In [9]:
# calculate accuracy
correct_preds_1 = df_merged_notNaN[df_merged_notNaN['matched_KF_column_1'] == df_merged_notNaN['GroundTruth Template Column']]
correct_preds_2 = df_merged_notNaN[df_merged_notNaN['matched_KF_column_2'] == df_merged_notNaN['GroundTruth Template Column']]
correct_preds_3 = df_merged_notNaN[df_merged_notNaN['matched_KF_column_3'] == df_merged_notNaN['GroundTruth Template Column']]

acc_top1 = len(correct_preds_1) / len(df_merged_notNaN)
acc_top2 = (len(correct_preds_1) + len(correct_preds_2)) / len(df_merged_notNaN)
acc_top3 = (len(correct_preds_1) + len(correct_preds_2) + len(correct_preds_3)) / len(df_merged_notNaN)

print(f'Top-1 Accuracy: {acc_top1:.4f}')
print(f'Top-2 Accuracy: {acc_top2:.4f}')
print(f'Top-3 Accuracy: {acc_top3:.4f}')

Top-1 Accuracy: 1.0000
Top-2 Accuracy: 1.0000
Top-3 Accuracy: 1.0000


100% accuracy for this competitor file. Need to check files 2-4 after.